In [239]:
saidas = {'conta1':115, 'conta2':424, 'fran':110}
entradas = {'mãe_credito': 110, 'conta':150, 'mesada':300, }

saida_total = sum(saidas.values())
entrada_total = sum(entradas.values())

print('saida:', saida_total)
print('entrada:', entrada_total)
print('diferença:', entrada_total-saida_total)

saida: 649
entrada: 560
diferença: -89


In [1]:
import numpy as np
import pandas as pd

# Perda de Carga

In [6]:
def Re(D, V, p=None, u=None, v=None):

    if v==None: Re = D * V * p / u
    else: Re = D * V / v
    
    return Re

In [8]:
def swamee_f (rug, D, Re):
    
    f = 0.25 * np.log10( rug / (3.7*D) + 5.74 / Re**0.9)**(-2)
    
    return f

In [4]:
def darcy_hp(L, f, V, D, g=9.81):
    
    hp = (L * f * V**2) / (D * 2 * g)
    
    return hp

In [192]:
def hazen(L, C, D, hp=None, Q=None):
    
    if hp==None: hp = L * ( 10.65 * Q**1.85 ) / ( C**1.85 * D**4.87 ); return hp
    if Q==None: Q = ( hp * ( C**1.85 * D**4.87 ) / (10.65 * L) ) ** (1/1.85); return Q

In [12]:
def formula_cinetica(V, g=9.81, hl=None, k=None):
    
    if hl==None: 
        hl = (k * V**2) / (2 * g)
        return hl
    else: 
        k = (hl * 2 * g) / (V**2)
        return k
        

# Condutos Equivalentes

In [22]:
def comp_equiv(Le=None, K=None, D=None, f=None):

    if Le==None: Le = K * D / f ; return Le
    if K==None: K = Le * f / D ; return K
    if D==None: D = Le * f / K ; return D
    if f==None: f = K * D / Le ; return f


### Condutos em Série

In [15]:
def darcy_serie(Li, Fi, Di, f=None, L=None, D=None):
    
    constante = sum([li * fi /  di**5 for li, fi, di in zip(Li, Fi, Di)])
    
    if L==None: L = constante * D**5 / f; return L
    if f==None: f = constante * D**5 / L; return f
    if D==None: D = (L * f / constante)**(1/5); return D
        

In [19]:
def hazen_serie(Li, Ci, Di, C=None, L=None, D=None):
    
    constante = sum([li / (ci**1.95 * di**4.87) for li, ci, di in zip(Li, Ci, Di)])
    
    if L==None: L = constante * D**4.87 * C**1.85; return L
    if C==None: C = ( L / (constante * D**4.87) )**(1/1.85); return C
    if D==None: D = ( L / (constante * C**1.85) )**(1/4.87); return D
        

### Condutos Paralelos

In [15]:
def darcy_paralelo(Li, Fi, Di, f=None, L=None, D=None):
    
    constante = sum([ di**2.5 / (fi*li)**(1/2) for li, fi, di in zip(Li, Fi, Di)])
    
    if L==None: L = D**5 / (f * constante**2); return L
    if f==None: f = D**5 / (L * constante**2); return f
    if D==None: D = (constante**2 * f * L)**(1/5); return D
        

In [18]:
def hazen_paralelo(Li, Ci, Di, C=None, L=None, D=None):
    
    constante = sum([ ci * di**2.63 / li**0.54 for li, ci, di in zip(Li, Ci, Di)])
    
    if L==None: L = ( D**2.63 * C / constante )**(1/0.54); return L
    if C==None: C = ( L * constante / D**2.63 ) ; return C
    if D==None: D = ( L**0.54 * constante / C )**(1/2.63); return D
    

# Conversões

In [23]:
def inches_to_meter(L):
    
    return L * 2.54 / 100

# Q7

In [227]:
p2 = 11 # mca = m
Q = 1.2e-3 * 8 # m3
C = 130
L1 = 50; L2 = 80
D12 = inches_to_meter(8)

Li = [120+20+20] * 8
Di = [inches_to_meter(1.5)] * 8
Ci = [C] * 8

def Q7 (p2, Q, C, L1, D12, Li, Di, Ci):

    Le = hazen_paralelo(Li=Li, Ci=Ci, Di=Di, C=C, D=D12)

    Lt = L1 + L2 + Le; Lt

    hp = hazen(C=C, L=Lt, D=D12, Q=Q); hp

    p1 = p2 + hp; p1
    
    print('Perda de Carga:', hp, '- Pressão 1:', p1, '- Pressão 2:', p2)

In [228]:
Q7 (p2, Q, C, L1, D12, Li, Di, Ci)

Perda de Carga: 6.782718783383077 - Pressão 1: 17.782718783383075 - Pressão 2: 11


# Q6

In [223]:
z1 = 54; z2 = 42; C=150; D=50e-3; 

Li = [1, 14, 6, 22]
Lei = [2.3, 3.2, 1.3, 1.3, 0.7, 3.2]


In [224]:
def Q6(z1, z2, C, D, Li, Lei):
    
    hp = z1-z2

    Lt = sum(Li) + sum(Lei)

    Q = hazen(L=Lt, C=C, hp=hp, D=D)
    
    print('Vazão:', Q)

In [225]:
Q6 (z1, z2, C, D, Li, Lei)

Vazão: 0.006894427606761677


# Q4

Uma instalação hidráulica com diâmetro de 50 mm e C=100 liga o reservatório A, cujo nível d'água (N.A.) está a 54 m, ao reservatório B, com N.A. em 44 m, tendo um comprimento total 43 m. Entre os dois, tem-se as seguintes perdas localizadas:

- entrada da tubulação: Leq = 0,7 m;

- saída da tubulação: Leq =  1,5 m;;

- um cotovelo 90°: Leq =  1,7 m;;

- duas curvas de 45°: Leq =  0,6 m; (cada);

- um registro de ângulo, aberto: Leq =  8,6 m;.

Determine, utilizando a equação de Hazen-Williams, a vazão transportada em regime permanente.

Obs.: Selecione a unidade desejada.

In [220]:
D = 60e-3; C=90; z1=54; z2=43; L=48

Lei = [0.9, 1.8, 2, 0.8, 0.8, 10.3]


In [221]:
def Q4 (D, C, z1, z2, L, Lei):
    
    hp = z1 - z2; print('hp:', hp)
    
    Lt = L + sum(Lei); print('Lt:', Lt, '- Le:', sum(Lei))
    
    Q = hazen(hp=hp, L=Lt, C=C, D=D)
    
    print('Vazão:', Q)

In [222]:
Q4 (D, C, z1, z2, L, Lei)

hp: 11
Lt: 64.6 - Le: 16.6
Vazão: 0.005846487456914062


# Q5

In [178]:
LA = [1]
LB = [0.5, 1, 1]

LAe = [0.3]
LBe = [1.5, 1.5]

Q = 0.6e-3 # m3

In [179]:
def Q5_horizontal(LA, LB, LAe, LBe, tubo='B'):
    
    LAt = sum(LA) + sum(LAe)
    LBt = sum(LB) + sum(LBe)
    
    dL = LAt - LBt
    
    if dL<0: dL*=-1
    print('Comprimento Equivalente do Acessório:') dL

In [180]:
Q5(LA, LB, LAe, LBe, tubo='B')

4.2